In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
class Net(nn.Module):

    def __init__(self, *args, **kwargs):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)


    def forward(self, input):
        a1 = F.relu(self.conv1(input))
        a1 = F.max_pool2d(a1, (2,2))

        a2 = F.relu(self.conv2(a1))
        a2 = F.max_pool2d(a2, (2,2))
        a2 = torch.flatten(a2)

        a3 = F.relu(self.fc1(a2))

        a4 = F.relu(self.fc2(a3))

        output = self.fc3(a4)

        return output
        

In [6]:
#Test Forward
net = Net()
x = torch.rand(1, 3, 32, 32)
output = net(x)
output

TypeError: descriptor '__init__' of 'super' object needs an argument